In [25]:
%reset -f
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nms import non_max_suppression_fast

In [26]:
basedir = "/media/kalfasyan/DATADRIVE1/sticky_plates/2020"
images_path = f"{basedir}/20200820/plates"
platename = "Herent_w29_1-90_F10_51 mm_ISO160_1-15 s_IncandescentWB.png"
imgpath = f"{images_path}/{platename}"

In [30]:
import glob

imgpaths = glob.glob(f"{images_path}/**jpg")

In [31]:
def overlay_image_nms(imgpath, plot_orig=True):
    image = cv2.imread(imgpath)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    gray = image[:,:,2] #cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    filtered = cv2.medianBlur(gray, 5) # 13
    # (thresh, blackAndWhiteImage) = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(filtered, 50, 200)

    # Highlighting detections
    (cnts,_) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edged_image = image.copy()
    cv2.drawContours(edged_image, cnts, -1, (0, 255, 0), 1);

    coordinates = []
    for cnt in cnts:
        M = cv2.moments(cnt)
        try:
            cx = int(M['m10']/M['m00'])
        except:
            cx = np.nan
        try:
            cy = int(M['m01']/M['m00'])
        except:
            cy = np.nan
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        if (12 < area) and (50 < perimeter):
            if np.nan not in [cx, cy]:
                x,y,w,h = cv2.boundingRect(cnt)
                edged_image = cv2.rectangle(edged_image, (x,y), (x+w, y+h), (0,255,0), 2)
                coordinates.append((x,y,x+w, y+h))

    print("{} insects counted in captured image.".format(len(coordinates)))
    if plot_orig:
        plt.figure(figsize=(30,24))
        plt.imshow(edged_image)

    # load the image and clone it
    boundingBoxes = np.array(coordinates)

    img = cv2.imread(imgpath)
#     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    orig = img.copy()

    # loop over the bounding boxes for each image and draw them
    for (startX, startY, endX, endY) in boundingBoxes:
        cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 0, 255), 2)

    # perform non-maximum suppression on the bounding boxes
    pick = non_max_suppression_fast(boundingBoxes, 0.3)
    # loop over the picked bounding boxes and draw them
    for (startX, startY, endX, endY) in pick:
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)

    for i, c in enumerate(pick):
        cv2.putText(img, "{}".format(i + 1), (c[0]-20, c[1]-10), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,250), 4)

    cv2.putText(img, platename, (10, 4150), cv2.FONT_HERSHEY_DUPLEX, 5., (0,0,250), 5)
    
#     cv2.imshow('IMAGE',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    cv2.imwrite(f"{imgpath[:-4]}_overlay.png",img)

    print(f"{len(pick)} images after non-maximal-suppression\n")
    return img

In [32]:
for imgpath in imgpaths:
    overlay_image_nms(imgpath, plot_orig=False)

772 insects counted in captured image.
754 images after non-maximal-suppression
524 insects counted in captured image.
489 images after non-maximal-suppression
619 insects counted in captured image.
581 images after non-maximal-suppression
871 insects counted in captured image.
810 images after non-maximal-suppression
623 insects counted in captured image.
591 images after non-maximal-suppression
439 insects counted in captured image.
406 images after non-maximal-suppression
1052 insects counted in captured image.
996 images after non-maximal-suppression
493 insects counted in captured image.
465 images after non-maximal-suppression
505 insects counted in captured image.
480 images after non-maximal-suppression
477 insects counted in captured image.
451 images after non-maximal-suppression
703 insects counted in captured image.
656 images after non-maximal-suppression
1101 insects counted in captured image.
1024 images after non-maximal-suppression
443 insects counted in captured image

In [5]:
result_image.shape

(4174, 6297, 3)

In [6]:
cv2.imshow('gete',result_image)
cv2.waitKey(0)

113